<a href="https://colab.research.google.com/github/divishagupta99/WiFi_Localisation/blob/master/WiFi_Localisation_Euclidean_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Preprocessing Data :
Git clone dataset from repository. 
Rename files to (*location_name*)_(*device number*). Eg. D156_2 means the location "D156" as recorded by device number 2. There are 4 such devices.
Import Dataframe and remove .csv extension.
Get rid of unused columns and datapoints of temporary APs.

In [ ]:
!git clone https://github.com/divishagupta99/WiFi_Localisation.git

fatal: destination path 'WiFi_Localisation' already exists and is not an empty directory.


In [ ]:
import os
def rename(): 
  i = 0  
  for filename in os.listdir("/content/WiFi_Localisation/Data Set/"): 
    dst =str(filename).replace('-','_')
    src ='/content/WiFi_Localisation/Data Set/'+ filename 
    dst ='/content/WiFi_Localisation/Data Set/'+ dst 
      
    # rename() function will 
    # rename all the files 
    os.rename(src, dst) 
    i += 1
  print(i)
  


In [ ]:
rename()

96


In [ ]:
dataframes = []
names = []
def importing_df():
  for filename in os.listdir("/content/WiFi_Localisation/Data Set/"):
    dataframes.append(pd.read_csv("/content/WiFi_Localisation/Data Set/"+str(filename),header=1))
    new_name = str(filename).replace('.csv','')
    names.append(new_name)


In [ ]:
def preprocessing():
  for i in range(len(dataframes)):
    #df_temp = dataframes[i]
    dataframes[i].drop(dataframes[i].loc[dataframes[i]['SSID']!= 'BPGC-NAB'].index, inplace=True)      #Considering only BPGC-NAB
    dataframes[i].reset_index(drop = True, inplace=True)
    for j in range(len(dataframes[i]['Strength'])):
      dataframes[i]['Strength'][j] = str(dataframes[i]['Strength'][j]).replace('dBm', '')
    dataframes[i]['Strength'] = dataframes[i]['Strength'].astype(int)
    dataframes[i] = dataframes[i].drop(columns = ['Primary Frequency', 'Distance', '802.11mc', 'Security', 'Time Stamp', 'SSID'])





In [ ]:
importing_df()
preprocessing()

In [ ]:
dataframes[0].head()

,BSSID,Strength
0,00:a3:8e:60:f2:a0,-76
1,08:4f:a9:10:4b:80,-74
2,70:ea:1a:22:8f:e0,-61
3,70:ea:1a:22:96:20,-81
4,70:ea:1a:22:96:a0,-74


### Unique Values in BSSID 

In [ ]:
BSSID = []
def unique_BSSID():
  for i in range(len(dataframes)):
    df_temp = dataframes[i]
    for j in df_temp['BSSID']:
      BSSID.append(j)

In [ ]:
unique_BSSID()

In [ ]:
BSSID = list(set(BSSID))

32 Unique MAC addresses

In [ ]:
#sorting BBSID
BSSID.sort()
BSSID
#The orderof BSSID will remain constant 

['00:a3:8e:4e:29:a0',
 '00:a3:8e:60:f2:a0',
 '08:4f:a9:10:4b:80',
 '08:4f:a9:2d:77:40',
 '08:4f:a9:2d:8a:40',
 '08:4f:a9:2d:8a:a0',
 '08:4f:a9:30:00:00',
 '08:4f:a9:39:68:20',
 '08:4f:a9:39:68:40',
 '08:4f:a9:39:6a:40',
 '08:4f:a9:43:38:a0',
 '68:3b:78:e9:c0:e0',
 '70:ea:1a:22:87:60',
 '70:ea:1a:22:8f:e0',
 '70:ea:1a:22:92:40',
 '70:ea:1a:22:96:00',
 '70:ea:1a:22:96:20',
 '70:ea:1a:22:96:a0',
 '70:ea:1a:22:97:80',
 '70:ea:1a:22:98:20',
 '70:ea:1a:22:9b:80',
 '70:ea:1a:22:9b:e0',
 '70:ea:1a:22:9f:20',
 '70:ea:1a:22:9f:80',
 '70:ea:1a:22:9f:e0',
 '70:ea:1a:22:a0:20',
 '70:ea:1a:22:a5:a0',
 '70:ea:1a:22:a9:40',
 '70:ea:1a:22:a9:a0',
 '70:ea:1a:22:ac:00',
 'a8:b4:56:ca:8f:40',
 'a8:b4:56:cc:de:00']

In [ ]:
dataframes[0]

,BSSID,Strength
0,00:a3:8e:60:f2:a0,-76
1,08:4f:a9:10:4b:80,-74
2,70:ea:1a:22:8f:e0,-61
3,70:ea:1a:22:96:20,-81
4,70:ea:1a:22:96:a0,-74
...,...,...
361,70:ea:1a:22:8f:e0,-60
362,70:ea:1a:22:96:20,-75
363,70:ea:1a:22:96:a0,-71
364,70:ea:1a:22:a9:a0,-80


## Fingerprint Creation 
Each fingerprint is a dictionary mapping of BSSIDs to Average Signal Strengths
Undetected APs have signal strengths as -95 dBm

In [ ]:
fingerprint_a = []
for j in range(len(dataframes)):
  df_temp = dataframes[j].copy()
  df_temp = df_temp.groupby('BSSID')
  mean_val = pd.DataFrame(df_temp['Strength'].mean())
  mean_val.to_csv('mean_val.csv', header = False)
  mean_val = pd.read_csv('mean_val.csv', header = None)
  mean_val.columns = ['BSSID', 'Avg_Signal_Strength']
  fingerprint = pd.DataFrame(columns = mean_val.columns)
  fingerprint['BSSID'] = BSSID
  for i in range(len(BSSID)):
    bssid_temp = fingerprint.iloc[i]['BSSID']
    if bssid_temp in list(mean_val['BSSID']):
      fingerprint.iloc[i]['Avg_Signal_Strength'] = list(mean_val[mean_val.BSSID == bssid_temp]['Avg_Signal_Strength'])[0]
    else:
      fingerprint.iloc[i]['Avg_Signal_Strength'] = -95            #Undetected APs, put -95dBm
  fingerprint['Avg_Signal_Strength'] = fingerprint['Avg_Signal_Strength'].astype(float)
  fingerprint_a.append(fingerprint)

In [ ]:
fingerprint_a[0]

,BSSID,Avg_Signal_Strength
0,00:a3:8e:4e:29:a0,-95.000000
1,00:a3:8e:60:f2:a0,-76.000000
2,08:4f:a9:10:4b:80,-64.266667
3,08:4f:a9:2d:77:40,-95.000000
4,08:4f:a9:2d:8a:40,-95.000000
5,08:4f:a9:2d:8a:a0,-95.000000
6,08:4f:a9:30:00:00,-95.000000
7,08:4f:a9:39:68:20,-95.000000
8,08:4f:a9:39:68:40,-95.000000
9,08:4f:a9:39:6a:40,-95.000000


BSSIDs are sorted, so the order remains same. Converting data to vector format. Selecting fingerprints from "_1.csv" and "_4.csv" only. "_2.csv" data is used for testing. 

In [ ]:
test = []
train = []
for i in range(len(fingerprint_a)):
  if names[i][-2:] == '_2':
    test.append(list([names[i] ,fingerprint_a[i].copy()]))
  elif names[i][-2:] == '_1' or names[i][-2:] == '_4':
    train.append(list([names[i], fingerprint_a[i].copy()]))
print(len(train))
print(len(test))


48
24


In [ ]:
train

[['D164_1',                 BSSID  Avg_Signal_Strength
  0   00:a3:8e:4e:29:a0           -82.571429
  1   00:a3:8e:60:f2:a0           -87.500000
  2   08:4f:a9:10:4b:80           -84.400000
  3   08:4f:a9:2d:77:40           -95.000000
  4   08:4f:a9:2d:8a:40           -55.877193
  5   08:4f:a9:2d:8a:a0           -67.033333
  6   08:4f:a9:30:00:00           -84.800000
  7   08:4f:a9:39:68:20           -49.283333
  8   08:4f:a9:39:68:40           -79.555556
  9   08:4f:a9:39:6a:40           -95.000000
  10  08:4f:a9:43:38:a0           -95.000000
  11  68:3b:78:e9:c0:e0           -95.000000
  12  70:ea:1a:22:87:60           -70.067797
  13  70:ea:1a:22:8f:e0           -95.000000
  14  70:ea:1a:22:92:40           -95.000000
  15  70:ea:1a:22:96:00           -84.000000
  16  70:ea:1a:22:96:20           -95.000000
  17  70:ea:1a:22:96:a0           -95.000000
  18  70:ea:1a:22:97:80           -95.000000
  19  70:ea:1a:22:98:20           -85.000000
  20  70:ea:1a:22:9b:80           -95.000000


# Finding Euclidean Distance
Calculate Euclidean Distance of a testing fingerprint with respect to every  fingerprint in training dataset. Take the least and assign its position

In [ ]:
def convert_vector(df):
  res = []
  for i in range(len(df)):
    res.append(df.iloc[i]['Avg_Signal_Strength'])
  return res

In [ ]:
for i in range(len(train)):
  train[i][1] = convert_vector(train[i][1])
for i in range(len(test)):
  test[i][1] = convert_vector(test[i][1])

In [ ]:
test_result = pd.DataFrame(columns = ['Original', 'Predicted'])
for j in range(len(test)):
  test_result.loc[j] = pd.Series({'Original' : test[j][0][:-2]})

In [ ]:
import math
def euclidean_distance(test, train):
  sum = 0
  for i in range(len(test)):
    sum = sum + (test[i] - train[i])**2
  return math.sqrt(sum)
  

In [ ]:
for i in range(len(test)):
  min_index = 0
  distance = np.inf
  for j in range(len(train)):
    ans = euclidean_distance(test[i][1], train[j][1])
    if(ans < distance):
      distance = ans
      min_index = j
  test_result.loc[i]['Predicted'] = train[min_index][0][:-2]



In [ ]:
test_result

,Original,Predicted
0,D267,D267
1,D250,D250
2,D262,D258
3,D168,D168
4,D269,D269
5,D160,D160
6,StairF,StairF
7,WF,EntranceF
8,D251,D251
9,D154,D154


In [ ]:
def match_adjacent(original_position, predicted_position):
  if (original_position=='WG' and (predicted_position == 'EntranceG' or predicted_position == 'D151')):
    return True
  elif (original_position=='D151' and (predicted_position == 'WG' or predicted_position == 'D153')):
    return True
  elif (original_position=='D153' and (predicted_position == 'D151' or predicted_position == 'D154')):
    return True
  elif (original_position=='D154' and (predicted_position == 'D153' or predicted_position == 'EEEG')):
    return True
  elif (original_position=='EEEG' and (predicted_position == 'D154' or predicted_position == 'StairG')):
    return True
  elif (original_position=='StairG' and (predicted_position == 'EEEG' or predicted_position == 'D156')):
    return True
  elif (original_position=='D156' and (predicted_position == 'StairG' or predicted_position == 'D160')):
    return True
  elif (original_position=='D160' and (predicted_position == 'D156' or predicted_position == 'D164')):
    return True
  elif (original_position=='D164' and (predicted_position == 'D160' or predicted_position == 'D168')):
    return True
  elif (original_position=='D168' and (predicted_position == 'D164' or predicted_position == 'D170')):
    return True
  elif (original_position=='D170' and (predicted_position == 'D168' or predicted_position == 'EntranceG')):
    return True
  elif (original_position=='EntranceG' and (predicted_position == 'D170' or predicted_position == 'WG')):
    return True

  elif (original_position=='EntranceF' and (predicted_position == 'WF' or predicted_position == 'D252')):
    return True
  elif (original_position=='WF' and (predicted_position == 'EntranceF' or predicted_position == 'D251')):
    return True
  elif (original_position=='D251' and (predicted_position == 'WF' or predicted_position == 'D250')):
    return True
  elif (original_position=='D250' and (predicted_position == 'D251' or predicted_position == 'D249')):
    return True
  elif (original_position=='D249' and (predicted_position == 'D250' or predicted_position == 'StairF')):
    return True
  elif (original_position=='StairF' and (predicted_position == 'D249' or predicted_position == 'D267')):
    return True
  elif (original_position=='D267' and (predicted_position == 'StairF' or predicted_position == 'D269' or predicted_position == 'D266')):
    return True
  elif (original_position=='D269' and (predicted_position == 'D267' or predicted_position == 'D266')):
    return True
  elif (original_position=='D266' and (predicted_position == 'D267' or predicted_position == 'D269' or predicted_position == 'D262')):
    return True
  elif (original_position=='D262' and (predicted_position == 'D266' or predicted_position == 'D258')):
    return True
  elif (original_position=='D258' and (predicted_position == 'D262' or predicted_position == 'D252')):
    return True
  elif (original_position=='D252' and (predicted_position == 'D258' or predicted_position == 'EntranceF')):
    return True
    
  else:
    return False



In [ ]:
def accuracy_score(df):
  sum_ans = 0
  for i in range(len(df)):
    if df['Original'][i] == df['Predicted'][i]:     #Original == Predicted exactly
      sum_ans = sum_ans + 2
    elif match_adjacent(df['Original'][i],(df['Predicted'][i])): #adjacent positions with lesser weights
      sum_ans = sum_ans + 1
    else:
      continue
  return sum_ans/(2*len(df))


In [ ]:
accuracy_score(test_result)

0.8333333333333334

In [ ]:
def exact_accuracy_score(df):
  sum_ans = 0
  for i in range(len(df)):
    if df['Original'][i] == df['Predicted'][i]:     #Original == Predicted exactly
      sum_ans = sum_ans + 1
  return sum_ans/(len(df))

exact_accuracy_score(test_result)

0.75

### Finding Euclidean Distance with Standardising each devices's values to it's mean and std deviation.

In [ ]:
import statistics
rssi_1 = []
rssi_2 =[]
rssi_4 = []

def normalise_dataframe():
  for i in range(len(dataframes)):
    if names[i][-2:]=='_3':
      continue  #Not considering this device due to avoid Android Throttling consequences
    for j in range(len(dataframes[i])):
      if names[i][-2:]=='_1':
        rssi_1.append(dataframes[i].loc[j,'Strength'])
      if names[i][-2:]=='_2':
        rssi_2.append(dataframes[i].loc[j,'Strength'])
      if names[i][-2:]=='_4':
        rssi_4.append(dataframes[i].loc[j,'Strength'])
  mean_1=statistics.mean(rssi_1)
  mean_2=statistics.mean(rssi_2)
  mean_4=statistics.mean(rssi_4)
  stddev_1 = statistics.stdev (rssi_1)
  stddev_2 = statistics.stdev(rssi_2)
  stddev_4 = statistics.stdev(rssi_4)
  print(mean_1, mean_2, mean_4, stddev_1, stddev_2, stddev_4)
  for i in range(len(dataframes)):
    for j in range(len(dataframes[i])):
      #print("Reached" + str(j))
      if names[i][-2:]=='_1':
        dataframes[i].loc[j,'Strength'] = ((dataframes[i].loc[j,'Strength'] - mean_1 )/(stddev_1) )
      if names[i][-2:]=='_2':
        dataframes[i].loc[j,'Strength'] = ((dataframes[i].loc[j,'Strength'] - mean_2 )/(stddev_2) )
      if names[i][-2:]=='_4':
        dataframes[i].loc[j,'Strength'] = ((dataframes[i].loc[j,'Strength'] - mean_4 )/(stddev_4) )

normalise_dataframe()

-74 -66 -64 9.591663046625438 12.288205727444508 11.789826122551595


In [ ]:
fingerprint_a = []
for j in range(len(dataframes)):
  df_temp = dataframes[j].copy()
  df_temp = df_temp.groupby('BSSID')
  mean_val = pd.DataFrame(df_temp['Strength'].mean())
  mean_val.to_csv('mean_val.csv', header = False)
  mean_val = pd.read_csv('mean_val.csv', header = None)
  mean_val.columns = ['BSSID', 'Avg_Signal_Strength']
  fingerprint = pd.DataFrame(columns = mean_val.columns)
  fingerprint['BSSID'] = BSSID
  for i in range(len(BSSID)):
    bssid_temp = fingerprint.iloc[i]['BSSID']
    if bssid_temp in list(mean_val['BSSID']):
      fingerprint.iloc[i]['Avg_Signal_Strength'] = list(mean_val[mean_val.BSSID == bssid_temp]['Avg_Signal_Strength'])[0]
    else:
      fingerprint.iloc[i]['Avg_Signal_Strength'] = -2            #Undetected APs, put -2 as least value
  fingerprint['Avg_Signal_Strength'] = fingerprint['Avg_Signal_Strength'].astype(float)
  fingerprint_a.append(fingerprint)

fingerprint_a[3]

,BSSID,Avg_Signal_Strength
0,00:a3:8e:4e:29:a0,-0.568296
1,00:a3:8e:60:f2:a0,-0.325515
2,08:4f:a9:10:4b:80,-2.000000
3,08:4f:a9:2d:77:40,-2.000000
4,08:4f:a9:2d:8a:40,-2.000000
5,08:4f:a9:2d:8a:a0,-2.000000
6,08:4f:a9:30:00:00,1.158539
7,08:4f:a9:39:68:20,-0.605379
8,08:4f:a9:39:68:40,0.602203
9,08:4f:a9:39:6a:40,1.538060


In [ ]:
test = []
train = []
for i in range(len(fingerprint_a)):
  if names[i][-2:] == '_2':
    test.append(list([names[i] ,fingerprint_a[i].copy()]))
  elif names[i][-2:] == '_1' or names[i][-2:] == '_4':
    train.append(list([names[i], fingerprint_a[i].copy()]))

for i in range(len(train)):
  train[i][1] = convert_vector(train[i][1])
for i in range(len(test)):
  test[i][1] = convert_vector(test[i][1])

test_result = pd.DataFrame(columns = ['Original', 'Predicted'])
for j in range(len(test)):
  test_result.loc[j] = pd.Series({'Original' : test[j][0][:-2]})

for i in range(len(test)):
  min_index = 0
  distance = np.inf
  for j in range(len(train)):
    ans = euclidean_distance(test[i][1], train[j][1])
    if(ans < distance):
      distance = ans
      min_index = j
  test_result.loc[i]['Predicted'] = train[min_index][0][:-2]

test_result

,Original,Predicted
0,D267,D267
1,D250,D250
2,D262,D258
3,D168,D168
4,D269,D269
5,D160,D160
6,StairF,StairF
7,WF,EntranceF
8,D251,D251
9,D154,D154


In [ ]:
accuracy_score(test_result)

0.9166666666666666

In [ ]:
exact_accuracy_score(test_result)

0.8333333333333334